***
<div  align='center'><img src='https://cdn-res.keymedia.com/cms/images/ca/155/0348_637304012816162034.jpg' width='25%'></div >
<h1><center> WebScraping project: Ottawa Housing Market </center></h1>
<h2><center>Created by: Ginta Grinfelde</center></h2> 

***

# Data Analysis

In [28]:
#Importing libraries
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go 
import plotly.express as px
import pandas as pd
import plotly.express as px

#Library for postal codes.
!pip install pypostalcode
from pypostalcode import PostalCodeDatabase

py.init_notebook_mode(connected=True)

Creating a new DataFrame 'properties' from a list of lists

In [10]:
#Importing the csv file

properties = pd.read_csv('point2homes.csv')
properties.head(5)

,prop_address,price,num_beds,num_baths,prop_type,prop_style,year_built,parking,basement,prop_taxes,neighborhood,postal_code,lot_info,assoc_fee,walk_score,transit_score,bike_score,latitude,longitude
0,"6705 CEDAR ACRES DR, Ottawa, Ontario, K4P1G4","$725,000 CAD",3,2,Single Family,Bungalow,Missing info,Missing info,N/A (Finished),Missing info,Greely,K4P1G4,Missing info,Missing info,2,5,38,45.2508202,-75.582962
1,"31 PALOMINO DRIVE N, Ottawa, Ontario, K2M1M2","$699,950 CAD",4,2,Single Family,Missing info,1985,Garage,Full (Finished),Missing info,Bridlewood - Emerald Meadows,K2M1M2,34.42ft x 104.92ft,Missing info,51,43,75,45.2946701,-75.8690872
2,"144 ANTHRACITE PRIVATE, Ottawa, Ontario, K2J7C8","$579,900 CAD",2,2,Single Family,Missing info,2022,Garage,None (Not Applicable),Missing info,Barrhaven,K2J7C8,21.00ft x 44.00ft,$120 CAD,16,Missing info,Missing info,45.2622337,-75.7473602
3,"497 MCLEOD Street, Ottawa, Ontario, K1R 5P9","$999,999 CAD",3,2,Residential,Detached,1930,Missing info,Missing info,Missing info,Centretown,K1R 5P9,Missing info,Missing info,92,71,93,45.409122,-75.699116
4,"468-470 BESSERER STREET, Ottawa, Ontario, K1N6C3","$3,325,000 CAD",2,Missing info,Multi-family,Missing info,1905,Missing info,Full (Partially Finished),Missing info,Sandy Hill,K1N6C3,Missing info,Missing info,93,85,99,45.4295616,-75.6811447


In [ ]:
properties.

## Data Cleaning

Replacing the 'Missing info' values with NaN.

In [11]:
properties = properties.replace("Missing info", np.nan)

Transforming 'price' to an integer to be able to use in calculations

In [12]:
properties['price'] = properties['price'].str.replace('$', '', regex = True).str.replace('CAD','', regex = True)\
.str.replace('K','', regex = True)\
.str.replace(' ','', regex = True)\
.str.replace(',','', regex = True)\
.replace('14/SF/YR', np.nan)\
.astype('Int64')

Transforming 'num_of_bedrooms' and 'num_bathrooms' to integers

In [13]:
properties['num_beds'] = properties['num_beds'].astype('Int64')
properties['num_baths'] = properties['num_baths'].astype('Int64')

Transforming neighborhood scores to integers

In [14]:
properties['walk_score'] = properties['walk_score'].astype('Int64')
properties['transit_score'] = properties['transit_score'].astype('Int64')
properties['bike_score'] = properties['bike_score'].astype('Int64')

Fixing property type column to the same format to have correct groups

In [15]:
properties['prop_type'] = properties['prop_type'].str.replace('MultiFamily', 'Multi-Family', regex = True)\
.str.replace('Multi-family', 'Multi-Family', regex = True)\
.str.replace('single family', 'Single Family', regex = True)

### Filtering
1. Upon data exploration, it was clear that some prices were not scraped properly so I filtered out properties which cost more than $10,000,000.

In [17]:
properties = properties[properties['price'] < 10000000]

2. Dropping lines where property types were not residential properties.

In [19]:
properties = properties[~properties['prop_type'].isin\
                        (['Vacant Land', 'lots/land' ,'Lots And Land' ,'Agriculture', 'Commercial'])]

3. The neighborhood list was too long to include in graphs so a different grouping method was needed.
    - Added a new column 'partial_postal_code' containing only the first three characters that denotes the city
    - Filtered out listings that do not have Ottawa postal code (do not start with K)
    - Created a new column with the new neighborhood names based on the postal code
    - Dropping neighborhoods that are far from central Ottawa

In [21]:
#Added a new column 'partial_postal_code' containing only the first three characters that denotes the city
properties['partial_post_code']= properties.postal_code.str[:3]

#Filtered out listings that do not have Ottawa postal code (do not start with K)
properties = properties[properties['partial_post_code'].str.contains('^K.*') == True]

In [26]:
#Created a new column with the new neighborhood names based on the postal code
pcdb = PostalCodeDatabase()
properties['new_neighb'] = [pcdb[code].city for code in properties['partial_post_code']]

In [27]:
#Dropping neighborhoods that are far from central Ottawa
properties = properties[~properties['new_neighb'].isin(['Perth', 'Carleton Place'\
            ,'South Leeds and Grenville United Counties (Prescott)', 'Rideau Lakes area (Kemptville)'\
            ,'Cumberland Township', 'Cumberland', 'Manotick', 'National Capital Region (Almonte)'])]

In [25]:
properties['partial_post_code']= properties['partial_post_code'].str.replace('K17', 'K1T', regex = True)

## Data Visualisation

###  Average house prices per neighborhood.


In [29]:
#Creating a bar chart to see the average property prices in Ottawa

avg_price = properties.groupby('new_neighb')['price'].mean().round().sort_values()

data = px.bar(x=avg_price.values, y=avg_price.index, orientation='h', text_auto='.2s'\
             ,title = 'Average house prices per neighborhood in Ottawa')

data.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False, marker_color='darkgreen')

data.update_layout(font_family='Droid Sans',
                  font_color ='black',
                  title_font_color= 'black',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis_title= 'Average sales price (CAD)',
                  yaxis_title='Neighborhood')

data.update_yaxes(dict(tickfont = dict(size=7.5)))
go.Figure(data)

### Average house prices per number of bedrooms

In [30]:
avg_price_num_bedrooms = properties.groupby('num_beds')['price'].mean().round().sort_values()

data = px.bar(x=avg_price_num_bedrooms.values, y=avg_price_num_bedrooms.index, orientation='h', text_auto='.2s'\
             ,title = 'Average house prices per number of bedrooms in Ottawa')

data.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False, marker_color='darkgreen')

data.update_layout(font_family='Droid Sans',
                  font_color ='black',
                  title_font_color= 'black',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis_title= 'Average sales price (CAD)',
                  yaxis_title='Number of Bedrooms')

data.update_yaxes(dict(tickfont = dict(size=15)))

go.Figure(data)

###  Average house price per property type and number of bedrooms

In [32]:
# Get the average prices per property type and num of bedrooms
avg_price_bedr_type = properties.groupby(['num_beds', 'prop_type'])['price'].mean().round()
print(avg_price_bedr_type)

num_beds  prop_type    
1         Condominium       395822.0
          Multi-Family      719957.0
          Residential       797073.0
          Single Family     437829.0
2         Condominium       508112.0
          Multi-Family     1536115.0
          Residential       654902.0
          Single Family     533054.0
3         Condominium       418562.0
          Residential       760427.0
          Single Family     685187.0
4         Condominium       360000.0
          Residential      1079924.0
          Single Family     995686.0
5         Residential      1348945.0
          Single Family    1207881.0
6         Residential      1787383.0
          Single Family    1492864.0
7         Residential      1500000.0
          Single Family    1536500.0
Name: price, dtype: Float64


In [33]:
import plotly.graph_objects as go

beds = list(set([x[0] for x in avg_price_bedr_type.index]))
colors = ['steelblue', 'turquoise', 'aquamarine', 'seagreen', 'mediumturquoise', 'mediumseagreen', 'green']

fig = go.Figure()

for b in beds:
    fig.add_trace(go.Bar(
        x=avg_price_bedr_type[b].index,
        y=avg_price_bedr_type[b].values,
        name= f'{b} beds',
        marker_color=colors[b-1]
    ))

fig.update_layout(barmode='group',
                  title= 'Average house price per property type and number of bedrooms',
                 font_family='Droid Sans',
                  font_color ='black',
                  plot_bgcolor='rgba(0,0,0,0)',
                  title_font_color= 'black',
                  xaxis_title= 'Property Type',
                  yaxis_title='Average Sales Price (CAD)')

fig.show()


### Mapping the top 10 most bikeable neighborhoods.


In [34]:
#Finding the top 10 neighborhoods that have the highest bike scores.
top_10 = properties.groupby('new_neighb')[['bike_score']].mean().nlargest(10,'bike_score')

In [35]:
#Joining tables to get the latitude and longitude for the neighborhoods
map_data = properties[['new_neighb', 'latitude', 'longitude']]\
            .drop_duplicates('new_neighb').set_index('new_neighb') \
            .join(top_10, how='inner')
map_data

,latitude,longitude,bike_score
new_neighb,,,
Ottawa (West Downtown area),45.409122,-75.699116,94.967213
Ottawa (Lower Town / Sandy Hill / University of Ottawa),45.4295616,-75.6811447,95.169014
Ottawa (Britannia / Pinecrest),45.3603592,-75.794487,84.470588
Ottawa (Vanier),45.4381714,-75.6757355,86.45098
Ottawa West,45.407089,-75.731964,94.196721
Ottawa (Westboro),45.3795013,-75.7264252,95.5
Ottawa (The Glebe / Ottawa South / Ottawa East),45.4007568,-75.7112885,95.454545
Ottawa (Parliament Hill),45.4220657,-75.6975861,94.1875
Ottawa (Centre Town),45.4068947,-75.716568,97.692308


In [36]:
#Creating a map of the top 10 most bikeable neighborhoods.
import folium

m = folium.Map(
    location=[45.4215, -75.6972],
    zoom_start=13
)

for x in map_data.iterrows():
    folium.Marker([x[1].latitude, x[1].longitude],
                 popup = x[0]).add_to(m)
m

### Mapping the top 10 neighborhoods with the best transit score

In [37]:
#Finding the top 10 neighborhoods that have the highest transit scores.
top_10_transit = properties.groupby('new_neighb')[['transit_score']].mean().nlargest(10,'transit_score')

In [38]:
#Joining tables to get the latitude and lingitude for the neighborhoods
map2_data = properties[['new_neighb', 'latitude', 'longitude']]\
            .drop_duplicates('new_neighb').set_index('new_neighb') \
            .join(top_10_transit, how='inner')
map2_data

,latitude,longitude,transit_score
new_neighb,,,
Ottawa (West Downtown area),45.409122,-75.699116,81.508197
Ottawa (Lower Town / Sandy Hill / University of Ottawa),45.4295616,-75.6811447,84.422535
Ottawa (Britannia / Pinecrest),45.3603592,-75.794487,62.176471
Ottawa West,45.407089,-75.731964,77.783333
Ottawa (Westboro),45.3795013,-75.7264252,64.9
Ottawa (The Glebe / Ottawa South / Ottawa East),45.4007568,-75.7112885,65.25
Ottawa (Highland Park / Carlingwood),45.3847389,-75.7602463,60.055556
Ottawa (Parliament Hill),45.4220657,-75.6975861,91.875
Ottawa (Alta Vista),45.3825302,-75.6490631,62.034483


In [39]:
#Creating the map
import folium

m = folium.Map(
    location=[45.4215, -75.6972],
    zoom_start=13
)

for x in map2_data.iterrows():
    folium.Marker([x[1].latitude, x[1].longitude],
                 popup = x[0]).add_to(m)
m    

### Mapping the top most walkable 10 neighborhoods 

In [40]:
#Finding the top 10 neighborhoods that have the highest bike scores.
top_10_walk = properties.groupby('new_neighb')[['walk_score']].mean().nlargest(10,'walk_score')

In [41]:
#Joining tables to get the latitude and lingitude for the neighborhoods
map3_data = properties[['new_neighb', 'latitude', 'longitude']]\
            .drop_duplicates('new_neighb').set_index('new_neighb') \
            .join(top_10_walk, how='inner')
map3_data

,latitude,longitude,walk_score
new_neighb,,,
Ottawa (West Downtown area),45.409122,-75.699116,87.508197
Ottawa (Lower Town / Sandy Hill / University of Ottawa),45.4295616,-75.6811447,89.042254
Ottawa (Vanier),45.4381714,-75.6757355,71.294118
Ottawa West,45.407089,-75.731964,74.295082
Ottawa (Westboro),45.3795013,-75.7264252,71.325
Ottawa (The Glebe / Ottawa South / Ottawa East),45.4007568,-75.7112885,72.909091
Ottawa (Highland Park / Carlingwood),45.3847389,-75.7602463,68.138889
Ottawa (Parliament Hill),45.4220657,-75.6975861,96.75
Ottawa (Centre Town),45.4068947,-75.716568,92.423077


In [42]:
#creating the map
import folium

m = folium.Map(
    location=[45.4215, -75.6972],
    zoom_start=13
)

for x in map3_data.iterrows():
    folium.Marker([x[1].latitude, x[1].longitude],
                 popup = x[0]).add_to(m)
m

### Finding neighborhoods that are most suitable for me

#### My wishes for buying a house:
- At least 3 bedrooms
- The neighborhood needs to have a walking score of more than 80
- The neighborhood needs to have a bike score of more than 80
- The neighborhood needs to have a transit score of more than 80
- The price ideally shouldn't be higher than 900,000 CAD

In [43]:
#Filtering properties that match my wishes and getting the names of the Neighborhoods
top_places = properties[(properties['num_beds'] >= 3) &
           (properties['walk_score'] >80) &
           (properties['bike_score'] >80) &
           (properties['transit_score'] >80) &
           (properties['price'] <= 900000)].drop_duplicates('new_neighb')

#Setting up map data
map_data4 = top_places[['new_neighb', 'latitude', 'longitude']].set_index('new_neighb')
map_data4

,latitude,longitude
new_neighb,,
Ottawa West,45.3983955,-75.7303009
Orleans (Fallingbrook),45.421143,-75.690057
Ottawa (West Downtown area),45.4180145,-75.7058182
Ottawa (Lower Town / Sandy Hill / University of Ottawa),45.4323997,-75.6898041
Ottawa (Centre Town),45.4154282,-75.6996002


In [44]:
import folium

m = folium.Map(
    location=[45.4215, -75.6972],
    zoom_start=13
)

for x in map_data4.iterrows():
    folium.Marker([x[1].latitude, x[1].longitude],
                 popup = x[0]).add_to(m)
m


### Using these top neighborhoods to find out the average price per bedroom

In [45]:
#Filtering the properties that are in the top neighborhoods.
best_n = properties[properties['new_neighb'].isin(top_places['new_neighb'])]

In [46]:
#Getting the average price per neighborhood per number of bedrooms.
avg_price_neighb = best_n.groupby(['num_beds', 'new_neighb'])['price'].mean().round()

In [47]:
#creating the graph
import plotly.graph_objects as go

beds = list(set([x[0] for x in avg_price_neighb.index]))
colors = ['steelblue', 'turquoise', 'aquamarine', 'seagreen', 'mediumturquoise', 'mediumseagreen', 'green']

fig = go.Figure()

for b in beds:
    fig.add_trace(go.Bar(
        x=avg_price_neighb[b].index,
        y=avg_price_neighb[b].values,
        name= f'{b} beds',
        marker_color=colors[b-1]
    ))

fig.update_layout(barmode='group',
                 font_family='Droid Sans',
                  title="Top Neighborhood average house prices",
                  title_font_size = 20,
                  font_color ='black',
                  font_size = 12,
                  plot_bgcolor='rgba(0,0,0,0)',
                  title_font_color= 'black',
                  xaxis_title= 'Neighborhood',
                  yaxis_title='Average Price')
fig.show()


# The End!